In [32]:
import json
import pandas as pd
import datetime
import os
from haversine import haversine

In [33]:
SFTP_store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190101-135843-638.txt",dtype=str,sep="|")
SFTP_store_list.shape

(1424, 13)

In [34]:
input_df=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2018_Q4/Planner_Request/SOTF_zip_20190107_Faith/2.8 Zip Codes_1 Stores Updated 1.7.19.xlsx",dtype=str)

print("4687" in SFTP_store_list['location_id'].tolist())

False


In [35]:
# 1851 SUNRISE HIGHWAY, BAY SHORE, NY, 11706
# Google from the store list from Dom
# Get 40.7397288,-73.2386392 from UI
input_df['Longitude_meas']=40.7397288
input_df['Latitude_meas']=-73.2386392

In [36]:
old_PST=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",dtype=str)
print("4687" in old_PST['location_id'].unique().tolist())

# New stores not in the TA

False


In [37]:
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))
zip_dist_dic={}
zip_list=[]
for zip_cd in zip_centers.keys():
    dist=haversine((40.7397288,-73.2386392),zip_centers[zip_cd],miles=True)
    if dist<=10:
        zip_list=zip_list+[zip_cd]
        zip_dist_dic.update({zip_cd:dist})


In [38]:
input_df=pd.concat([input_df]*len(zip_list))

In [39]:
input_df['Revenue_flag']="new_in_10_miles"
input_df['Zip']=zip_list
input_df['Store_Zip']="11706"
input_df['Distance (mile)']=input_df['Zip'].apply(lambda x: zip_dist_dic[x])

In [40]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2018_Q4/Planner_Request/SOTF_zip_20190107_Faith/BL_2.8 Zip Codes_1 Stores Updated 1.7.19_JL_20190107.xlsx",engine="xlsxwriter")
input_df.to_excel(writer,"SOTF_4687")
writer.save()